In [101]:
import pandas as pd

In [102]:
df_nazwy = pd.read_csv('Dane/nazwy.csv', sep=';')
df_nazwy = df_nazwy.drop(columns=['id'])
df_nazwy = df_nazwy.sort_values(by='kod')
df_nazwy = df_nazwy.reset_index(drop=True)
print(df_nazwy.loc[174:])


           nazwa      kod
174    Ostrołęka  1461000
175        Płock  1462000
176        Radom  1463000
177      Siedlce  1464000
178     Warszawa  1465000
..           ...      ...
375      wałecki  3217000
376      łobeski  3218000
377     Koszalin  3261000
378     Szczecin  3262000
379  Świnoujście  3263000

[206 rows x 2 columns]


In [103]:
df_wyniki = pd.read_csv('Dane/2023_Sejmowe.csv', sep=';')
df_wyniki['TERYT Powiatu'] = pd.to_numeric(df_wyniki['TERYT Powiatu'], errors='coerce')
df_wyniki = df_wyniki.dropna(subset=['TERYT Powiatu'])

df_wyniki['TERYT Powiatu'] = df_wyniki['TERYT Powiatu'].astype(int)
df_wyniki['TERYT Powiatu'] *= 10
print(df_wyniki.head())


# Konwersja df_nazwy['kod'] na zbiór dla szybszego wyszukiwania
nazwy_kody = set(df_nazwy['kod'])

# Sprawdzamy brakujące kody
brakujace_kody = [val for val in df_wyniki['TERYT Powiatu'] if val not in nazwy_kody]

# Wypisujemy brakujące wartości, jeśli istnieją
if brakujace_kody:
    print("BRAK kodów:", brakujace_kody)
else:
    print("Wszystkie kody są dopasowane.")
print(len(nazwy_kody))


   TERYT Powiatu          Powiat   Województwo  Nr okręgu  Liczba komisji  \
0         201000   bolesławiecki  dolnośląskie          1              82   
1         202000  dzierżoniowski  dolnośląskie          2              90   
2         203000       głogowski  dolnośląskie          1              57   
3         204000        górowski  dolnośląskie          3              29   
4         205000        jaworski  dolnośląskie          1              47   

   Liczba uwzględnionych komisji  Komisja otrzymała kart do głosowania  \
0                             82                                 62054   
1                             90                                 69081   
2                             57                                 62314   
3                             29                                 23738   
4                             47                                 35087   

   Liczba wyborców uprawnionych do głosowania  \
0                                       663

In [104]:
df_wyniki = df_wyniki.drop(columns=df_wyniki.columns[-6:])

In [105]:
df_wyniki.rename(columns={
    'TERYT Powiatu': 'Kod',
    'Liczba wyborców uprawnionych do głosowania': 'Uprawnieni do głosowania',
    'KOMITET WYBORCZY BEZPARTYJNI SAMORZĄDOWCY': 'Bezpartyjni Samorządowcy',
    'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 'Trzecia Droga',
    'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ': 'Konfederacja',
    'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI': 'Koalicja Obywatelska',
    'KOMITET WYBORCZY NOWA LEWICA': 'Lewica',
    'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 'Prawo i Sprawiedliwość',
    'Liczba kart ważnych': 'Ważnych kart',
    'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów': 'Oddane głosy'
}, inplace=True)

kolumny_do_usuniecia = list(df_wyniki.columns[:29])
kolumny_do_zachowania = ['Kod', 'Uprawnieni do głosowania', 'Ważnych kart', 'Oddane głosy']
kolumny = [col for col in df_wyniki.columns if col not in kolumny_do_usuniecia or col in kolumny_do_zachowania]
kolumny.remove('Bezpartyjni Samorządowcy')
del kolumny_do_zachowania, kolumny_do_usuniecia
df_wyniki = df_wyniki[kolumny]
del kolumny

In [106]:
kolumny = df_wyniki.columns
kolumny = kolumny.to_list()
kolumny.extend(['Data', 'Typ wyborów'])
df = df_wyniki.copy()

df = df_wyniki.assign(Data='2023-10-15', Typ_wyborów='Sejmowe')


In [107]:
df_wyniki = pd.read_csv('Dane/2020_Prezydenckie.csv', sep=';')
kolumny_do_usuniecia = list(df_wyniki.columns[:26])
kolumny_do_zachowania = ['Kod TERYT', 'Liczba wyborców uprawnionych do głosowania', 'Liczba kart ważnych', 'Liczba głosów ważnych oddanych łącznie na wszystkich kandydatów']
kolumny = [col for col in df_wyniki.columns if col not in kolumny_do_usuniecia or col in kolumny_do_zachowania]
kolumny = kolumny[:-3]
print(kolumny)

['Kod TERYT', 'Liczba wyborców uprawnionych do głosowania', 'Liczba kart ważnych', 'Liczba głosów ważnych oddanych łącznie na wszystkich kandydatów', 'Robert BIEDROŃ', 'Krzysztof BOSAK', 'Andrzej Sebastian DUDA', 'Szymon Franciszek HOŁOWNIA', 'Marek JAKUBIAK', 'Władysław Marcin KOSINIAK-KAMYSZ', 'Mirosław Mariusz PIOTROWSKI', 'Paweł Jan TANAJNO', 'Rafał Kazimierz TRZASKOWSKI']


In [108]:
kolumny.remove('Paweł Jan TANAJNO')
kolumny.remove('Marek JAKUBIAK')
kolumny.remove('Mirosław Mariusz PIOTROWSKI')

In [109]:
df_wyniki = df_wyniki[kolumny]
del kolumny, kolumny_do_usuniecia, kolumny_do_zachowania
df_wyniki.rename(columns={
    'Kod TERYT': 'Kod',
    'Liczba wyborców uprawnionych do głosowania': 'Uprawnieni do głosowania',
    'Liczba głosów ważnych oddanych łącznie na wszystkich kandydatów': 'Oddane głosy',
    'Liczba kart ważnych': 'Ważnych kart',
    'Krzysztof BOSAK': 'Konfederacja',
    'Andrzej Sebastian DUDA': 'Prawo i Sprawiedliwość',
    'Szymon Franciszek HOŁOWNIA': 'Polska 2050',
    'Władysław Marcin KOSINIAK-KAMYSZ': 'Polskie Stronnictwo Ludowe',
    'Rafał Kazimierz TRZASKOWSKI': 'Koalicja Obywatelska',
    'Robert BIEDROŃ': 'Lewica'
}, inplace=True)

In [110]:
print(df_wyniki.head())
df_wyniki['Trzecia Droga'] = df_wyniki['Polska 2050'] + df_wyniki['Polskie Stronnictwo Ludowe']
df_wyniki = df_wyniki.drop(columns=['Polskie Stronnictwo Ludowe', 'Polska 2050'])
print(df_wyniki.head())

     Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  Lewica  \
0  20100                     68554         41194         41035     852   
1  20200                     78876         46499         46340    1002   
2  20300                     67357         42143         42019     980   
3  20400                     27089         14642         14602     292   
4  20500                     39708         23406         23314     479   

   Konfederacja  Prawo i Sprawiedliwość  Polska 2050  \
0          2759                   17053         6726   
1          2824                   20403         6336   
2          2499                   18568         5969   
3           906                    7307         1918   
4          1528                   11139         2607   

   Polskie Stronnictwo Ludowe  Koalicja Obywatelska  
0                         633                 12732  
1                         687                 14770  
2                         736                 13000  
3 

In [111]:
df_wyniki['Kod'] = df_wyniki['Kod'].astype(int)
df_wyniki['Kod'] *= 10
print(df_wyniki.head())


# Konwersja df_nazwy['kod'] na zbiór dla szybszego wyszukiwania
nazwy_kody = set(df_nazwy['kod'])

# Sprawdzamy brakujące kody
brakujace_kody = [val for val in df_wyniki['Kod'] if val not in nazwy_kody]

# Wypisujemy brakujące wartości, jeśli istnieją
if brakujace_kody:
    print("BRAK kodów:", brakujace_kody)
else:
    print("Wszystkie kody są dopasowane.")
print(len(nazwy_kody))


      Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  Lewica  \
0  201000                     68554         41194         41035     852   
1  202000                     78876         46499         46340    1002   
2  203000                     67357         42143         42019     980   
3  204000                     27089         14642         14602     292   
4  205000                     39708         23406         23314     479   

   Konfederacja  Prawo i Sprawiedliwość  Koalicja Obywatelska  Trzecia Droga  
0          2759                   17053                 12732           7359  
1          2824                   20403                 14770           7023  
2          2499                   18568                 13000           6705  
3           906                    7307                  3722           2296  
4          1528                   11139                  6708           3324  
Wszystkie kody są dopasowane.
380


In [112]:
df_wyniki = df_wyniki.assign(Data='2020-06-28', Typ_wyborów='Prezydenckie')
df = pd.concat([df, df_wyniki], ignore_index=True)

print(df.sort_values(by='Kod').head())

        Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  \
0    201000                     66394         46510         45479   
380  201000                     68554         41194         41035   
381  202000                     78876         46499         46340   
1    202000                     74042         50902         49867   
2    203000                     64410         47421         46604   

     Trzecia Droga  Lewica  Prawo i Sprawiedliwość  Konfederacja  \
0             5339    4146                   15465          3425   
380           7359     852                   17053          2759   
381           7023    1002                   20403          2824   
1             6041    3797                   17865          3582   
2             4417    4120                   17212          2851   

     Koalicja Obywatelska        Data   Typ_wyborów  
0                   14329  2023-10-15       Sejmowe  
380                 12732  2020-06-28  Prezydenckie  
381           

In [113]:
df_wyniki = pd.read_csv('Dane/2019_Sejmowe.csv', sep=';')

kolumny_do_zachowania = ['Kod TERYT', 'Liczba wyborców uprawnionych do głosowania', 'Liczba kart ważnych', 'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów']
kolumny_do_usuniecia = list(df_wyniki.columns[:25])
kolumny = [col for col in df_wyniki.columns if col not in kolumny_do_usuniecia or col in kolumny_do_zachowania]
kolumny.remove('KOMITET WYBORCZY AKCJA ZAWIEDZIONYCH EMERYTÓW RENCISTÓW - ZPOW-601-21/19')
kolumny.remove('KOMITET WYBORCZY PRAWICA - ZPOW-601-20/19')
kolumny.remove('KOMITET WYBORCZY SKUTECZNI PIOTRA LIROYA-MARCA - ZPOW-601-17/19')
kolumny.remove('KOMITET WYBORCZY WYBORCÓW KOALICJA BEZPARTYJNI I SAMORZĄDOWCY - ZPOW-601-10/19')
kolumny.remove('KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19')
df_wyniki = df_wyniki[kolumny]


In [114]:
df_wyniki.rename(columns={
    'Kod TERYT': 'Kod',
    'Liczba wyborców uprawnionych do głosowania': 'Uprawnieni do głosowania',
    'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów': 'Oddane głosy',
    'Liczba kart ważnych': 'Ważnych kart',
    'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19': 'Koalicja Obywatelska',
    'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19': 'Konfederacja',
    'KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19': 'Trzecia Droga',
    'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19': 'Prawo i Sprawiedliwość',
    'KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19': 'Lewica'
}, inplace=True)
print(df_wyniki.columns)

Index(['Kod', 'Uprawnieni do głosowania', 'Ważnych kart', 'Oddane głosy',
       'Koalicja Obywatelska', 'Konfederacja', 'Trzecia Droga',
       'Prawo i Sprawiedliwość', 'Lewica'],
      dtype='object')


In [115]:
df_wyniki['Kod'] = df_wyniki['Kod'].astype(int)
df_wyniki['Kod'] *= 10
print(df_wyniki.head())


# Konwersja df_nazwy['kod'] na zbiór dla szybszego wyszukiwania
nazwy_kody = set(df_nazwy['kod'])

# Sprawdzamy brakujące kody
brakujace_kody = [val for val in df_wyniki['Kod'] if val not in nazwy_kody]

# Wypisujemy brakujące wartości, jeśli istnieją
if brakujace_kody:
    print("BRAK kodów:", brakujace_kody)
else:
    print("Wszystkie kody są dopasowane.")
print(len(nazwy_kody))


      Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  \
0  201000                     69098         38608         38018   
1  202000                     80196         44431         43827   
2  203000                     68354         41193         40741   
3  204000                     27176         13695         13478   
4  205000                     39924         21941         21600   

   Koalicja Obywatelska  Konfederacja  Trzecia Droga  Prawo i Sprawiedliwość  \
0                  7803          2558           2741                   16136   
1                 11618          2473           4612                   19113   
2                 10657          2154           2066                   18041   
3                  3044           749           1090                    6581   
4                  4623          1164           2208                   10364   

   Lewica  
0    6716  
1    4897  
2    6799  
3    1518  
4    2865  
Wszystkie kody są dopasowane.
380


In [116]:
df_wyniki = df_wyniki.assign(Data='2019-10-13', Typ_wyborów='Sejmowe')
df = pd.concat([df, df_wyniki], ignore_index=True)

In [117]:
df_wyniki = pd.read_excel('Dane/2018_Samorządowe.xlsx')


kolumny_do_zachowania = ['TERYT', 'Liczba wyborców uprawnionych do głosowania', 'Liczba kart ważnych', 'Liczba głosów ważnych']
kolumny_do_usuniecia = list(df_wyniki.columns[:26])

kolumny = [col for col in df_wyniki.columns if col not in kolumny_do_usuniecia or col in kolumny_do_zachowania]
df_wyniki = df_wyniki[kolumny[:-18]]

In [118]:
df_wyniki = df_wyniki.drop(columns=["KWW KUKIZ'15", "KWW BEZPARTYJNI SAMORZĄDOWCY"])
df_wyniki.rename(columns={
    'TERYT': 'Kod',
    'Liczba wyborców uprawnionych do głosowania': 'Uprawnieni do głosowania',
    'Liczba głosów ważnych': 'Oddane głosy',
    'Liczba kart ważnych': 'Ważnych kart',
    'KW PRAWO I SPRAWIEDLIWOŚĆ': 'Prawo i Sprawiedliwość',
    'KKW PLATFORMA.NOWOCZESNA KOALICJA OBYWATELSKA': 'Koalicja Obywatelska',
    'KOMITET WYBORCZY PSL': 'Trzecia Droga',
    'KKW SLD LEWICA RAZEM': 'Lewica',
    'KW RAZEM': 'Razem',
    'KWW WOLNOŚĆ W SAMORZĄDZIE': 'Wolność',
    'KW RUCH NARODOWY RP': 'Ruch Narodowy',
}, inplace=True)

print(df_wyniki.columns)

Index(['Kod', 'Uprawnieni do głosowania', 'Ważnych kart', 'Oddane głosy',
       'Prawo i Sprawiedliwość', 'Koalicja Obywatelska', 'Trzecia Droga',
       'Lewica', 'Wolność', 'Razem', 'Ruch Narodowy', 'KW PARTIA ZIELONI'],
      dtype='object')


In [119]:
df_wyniki['Konfederacja'] = df_wyniki['Wolność'].fillna(0) + df_wyniki['Ruch Narodowy'].fillna(0)

df_wyniki['Lewica'] = df_wyniki['Razem'] + df_wyniki['Lewica']
df_wyniki['Koalicja Obywatelska'] = df_wyniki['Koalicja Obywatelska'] + df_wyniki['KW PARTIA ZIELONI'].fillna(0)
df_wyniki = df_wyniki.drop(columns=['Wolność', 'Ruch Narodowy', 'KW PARTIA ZIELONI', 'Razem'])

In [120]:
df_wyniki['Kod'] = df_wyniki['Kod'].astype(int)
df_wyniki['Kod'] *= 1000
print(df_wyniki.head())


# Konwersja df_nazwy['kod'] na zbiór dla szybszego wyszukiwania
nazwy_kody = set(df_nazwy['kod'])

# Sprawdzamy brakujące kody
brakujace_kody = [val for val in df_wyniki['Kod'] if val not in nazwy_kody]

# Wypisujemy brakujące wartości, jeśli istnieją
if brakujace_kody:
    print("BRAK kodów:", brakujace_kody)
else:
    print("Wszystkie kody są dopasowane.")
print(len(nazwy_kody))


      Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  \
0  201000                     70600         36148         32887   
1  202000                     82041         40891         37752   
2  203000                     70005         37021         34576   
3  204000                     27893         14052         12086   
4  205000                     41084         22786         20555   

   Prawo i Sprawiedliwość  Koalicja Obywatelska  Trzecia Droga  Lewica  \
0                    7961                3065.0           1180    1984   
1                   12308                9810.0           1756    3149   
2                   11627                9144.0           1242    3821   
3                    4083                3031.0            927     664   
4                    6230                4003.0           3502    1318   

   Konfederacja  
0         944.0  
1        1290.0  
2         677.0  
3         345.0  
4         454.0  
Wszystkie kody są dopasowane.
380


In [121]:
df_wyniki = df_wyniki.assign(Data='2018-06-30', Typ_wyborów='Samorządowe')
df = pd.concat([df, df_wyniki], ignore_index=True)

print(df.sort_values(by='Kod').head())

         Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  \
0     201000                     66394         46510         45479   
760   201000                     69098         38608         38018   
1140  201000                     70600         36148         32887   
380   201000                     68554         41194         41035   
1     202000                     74042         50902         49867   

      Trzecia Droga  Lewica  Prawo i Sprawiedliwość  Konfederacja  \
0              5339    4146                   15465        3425.0   
760            2741    6716                   16136        2558.0   
1140           1180    1984                    7961         944.0   
380            7359     852                   17053        2759.0   
1              6041    3797                   17865        3582.0   

      Koalicja Obywatelska        Data   Typ_wyborów  
0                  14329.0  2023-10-15       Sejmowe  
760                 7803.0  2019-10-13       Sejmowe  

In [122]:
df_wyniki = pd.read_excel('Dane/2024_Samorządowe.xlsx')


kolumny_do_usuniecia = list(df_wyniki.columns[:27])
kolumny_do_zachowania = ['TERYT Powiatu', 'Liczba wyborców uprawnionych do głosowania', 'Liczba kart ważnych', 'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów']
kolumny = [col for col in df_wyniki.columns if col not in kolumny_do_usuniecia or col in kolumny_do_zachowania]
kolumny = kolumny[:-20]
kolumny.remove('KOMITET WYBORCZY NORMALNY KRAJ')
kolumny.remove('KOMITET WYBORCZY POLSKA LIBERALNA STRAJK PRZEDSIĘBIORCÓW')
kolumny.remove('KOMITET WYBORCZY INICJATYWA OBYWATELSKA POWIATU TARNOGÓRSKIEGO')
kolumny.remove('KOMITET WYBORCZY KOALICJA SAMORZĄDOWA OK SAMORZĄD')
kolumny.append('KOMITET WYBORCZY WYBORCÓW KONFEDERACJA I BEZPARTYJNI SAMORZĄDOWCY')
df_wyniki = df_wyniki[kolumny]

In [123]:
df_wyniki.rename(columns={
    'TERYT Powiatu': 'Kod',
    'Liczba wyborców uprawnionych do głosowania': 'Uprawnieni do głosowania',
    'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów': 'Oddane głosy',
    'Liczba kart ważnych': 'Ważnych kart',
    'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA': 'Koalicja Obywatelska',
    'KOMITET WYBORCZY WYBORCÓW KONFEDERACJA I BEZPARTYJNI SAMORZĄDOWCY': 'Konfederacja',
    'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 'Trzecia Droga',
    'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 'Prawo i Sprawiedliwość',
    'KOALICYJNY KOMITET WYBORCZY LEWICA': 'Lewica'
},inplace=True)

In [124]:
df_wyniki['Kod'] = df_wyniki['Kod'].astype(int)
df_wyniki['Kod'] *= 10
print(df_wyniki.head())


# Konwersja df_nazwy['kod'] na zbiór dla szybszego wyszukiwania
nazwy_kody = set(df_nazwy['kod'])

# Sprawdzamy brakujące kody
brakujace_kody = [val for val in df_wyniki['Kod'] if val not in nazwy_kody]

# Wypisujemy brakujące wartości, jeśli istnieją
if brakujace_kody:
    print("BRAK kodów:", brakujace_kody)
else:
    print("Wszystkie kody są dopasowane.")
print(len(nazwy_kody))


      Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  \
0  201000                     67774         32626         30749   
1  202000                     75472         35101         33272   
2  203000                     66107         31646         30376   
3  204000                     26261         13509         12204   
4  205000                     38267         19895         18672   

   Koalicja Obywatelska  Lewica  Trzecia Droga  Prawo i Sprawiedliwość  \
0                  5371    1911           4269                    8074   
1                 10806    2134           4288                   10873   
2                  9235    2329           3559                   10721   
3                  2120     664           1171                    3454   
4                  4799     994           2418                    6571   

   Konfederacja  
0          1710  
1          2366  
2          1832  
3           813  
4          1526  
Wszystkie kody są dopasowane.
380


In [125]:
df_wyniki = df_wyniki.assign(Data='2024-04-07', Typ_wyborów='Samorządowe')
df = pd.concat([df, df_wyniki], ignore_index=True)

print(df.sort_values(by='Kod').head())

         Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  \
0     201000                     66394         46510         45479   
1520  201000                     67774         32626         30749   
1140  201000                     70600         36148         32887   
760   201000                     69098         38608         38018   
380   201000                     68554         41194         41035   

      Trzecia Droga  Lewica  Prawo i Sprawiedliwość  Konfederacja  \
0              5339    4146                   15465        3425.0   
1520           4269    1911                    8074        1710.0   
1140           1180    1984                    7961         944.0   
760            2741    6716                   16136        2558.0   
380            7359     852                   17053        2759.0   

      Koalicja Obywatelska        Data   Typ_wyborów  
0                  14329.0  2023-10-15       Sejmowe  
1520                5371.0  2024-04-07   Samorządowe  

In [126]:
df_wyniki = pd.read_csv('Dane/2024_Europarlament.csv', sep=';')

kolumny_do_zachowania = ['TERYT Powiatu', 'Liczba wyborców uprawnionych do głosowania', 'Liczba kart ważnych', 'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów']
kolumny_do_usuniecia = list(df_wyniki.columns[:33])
kolumny = [col for col in df_wyniki.columns[:-3] if col not in kolumny_do_usuniecia or col in kolumny_do_zachowania]
kolumny.remove('KOMITET WYBORCZY POLEXIT')
kolumny.remove('KOMITET WYBORCZY POLSKA LIBERALNA STRAJK PRZEDSIĘBIORCÓW')
kolumny.remove('KOMITET WYBORCZY NORMALNY KRAJ')
df_wyniki = df_wyniki[kolumny]

In [127]:
df_wyniki.rename(columns={
    'TERYT Powiatu': 'Kod',
    'Liczba wyborców uprawnionych do głosowania': 'Uprawnieni do głosowania',
    'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów': 'Oddane głosy',
    'Liczba kart ważnych': 'Ważnych kart',
    'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA': 'Koalicja Obywatelska',
    'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ': 'Konfederacja',
    'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 'Trzecia Droga',
    'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 'Prawo i Sprawiedliwość',
    'KOALICYJNY KOMITET WYBORCZY LEWICA': 'Lewica'
}, inplace=True)
print(df_wyniki.columns)

Index(['Kod', 'Uprawnieni do głosowania', 'Ważnych kart', 'Oddane głosy',
       'Koalicja Obywatelska', 'Lewica', 'Trzecia Droga', 'Konfederacja',
       'Prawo i Sprawiedliwość'],
      dtype='object')


In [128]:
df_wyniki['Kod'] = df_wyniki['Kod'].astype(int)
df_wyniki['Kod'] *= 10
print(df_wyniki.head())


# Konwersja df_nazwy['kod'] na zbiór dla szybszego wyszukiwania
nazwy_kody = set(df_nazwy['kod'])

# Sprawdzamy brakujące kody
brakujace_kody = [val for val in df_wyniki['Kod'] if val not in nazwy_kody]

# Wypisujemy brakujące wartości, jeśli istnieją
if brakujace_kody:
    print("BRAK kodów:", brakujace_kody)
else:
    print("Wszystkie kody są dopasowane.")
print(len(nazwy_kody))


      Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  \
0  201000                     67127         23710         23514   
1  202000                     74455         27881         27676   
2  203000                     65168         25179         25035   
3  204000                     25951          7742          7675   
4  205000                     37839         13545         13443   

   Koalicja Obywatelska  Lewica  Trzecia Droga  Konfederacja  \
0                  8472    1528           1123          3176   
1                 10524    1675           1486          3095   
2                  8641    2808           1185          2972   
3                  2409     606            367          1010   
4                  4964     692            608          1610   

   Prawo i Sprawiedliwość  
0                    8845  
1                   10509  
2                    9102  
3                    3169  
4                    5398  
Wszystkie kody są dopasowane.
380


In [129]:
df_wyniki = df_wyniki.assign(Data='2024-06-09', Typ_wyborów='Europarlament')
df = pd.concat([df, df_wyniki], ignore_index=True)

print(df.sort_values(by='Kod').head())

         Kod  Uprawnieni do głosowania  Ważnych kart  Oddane głosy  \
0     201000                     66394         46510         45479   
1140  201000                     70600         36148         32887   
380   201000                     68554         41194         41035   
1520  201000                     67774         32626         30749   
1900  201000                     67127         23710         23514   

      Trzecia Droga  Lewica  Prawo i Sprawiedliwość  Konfederacja  \
0              5339    4146                   15465        3425.0   
1140           1180    1984                    7961         944.0   
380            7359     852                   17053        2759.0   
1520           4269    1911                    8074        1710.0   
1900           1123    1528                    8845        3176.0   

      Koalicja Obywatelska        Data    Typ_wyborów  
0                  14329.0  2023-10-15        Sejmowe  
1140                3065.0  2018-06-30    Samorządow

In [130]:
df.rename(columns={
    'Typ_wyborów': 'Typ wyborów'
}, inplace=True)

In [131]:
df['Koalicja Obywatelska'] = df['Koalicja Obywatelska'].astype(int)
df['Konfederacja'] = df['Konfederacja'].astype(int)

In [ ]:
df.to_csv('Dane/Wyniki.csv', sep=';', encoding='utf-8-sig', index=False, )